This is the main notebook for taking a csv of prompts, getting AI responses, then parsing with prosodic.py

In [ ]:
# install requirements (replace this later with requirements.txt)

## run this cell to install prosodic, then restart runtime as requested

%pip install -U prosodic
#%pip install openai


In [9]:
# set api keys
import os
os.environ["OPENAI_API_KEY"] = "ADD"
os.environ["GEMINI_API_KEY"] = "ADD"


In [ ]:
#openai api
#%pip install openai

import pandas as pd

# Read the prompt_dict CSV file into a DataFrame
df_prompt_dict = pd.read_csv('/Users/bsg25/metricalgpt/prompt_dict.csv') # or replace file path with your own

# Convert the DataFrame to a dictionary
gen_prompt_dict = df_prompt_dict.to_dict(orient='records')

import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_KEY"),  # replace with your API key
)

# Assuming you have set your OpenAI API key

for prompt_dict in gen_prompt_dict:
    try:
        # Create completion using OpenAI API
        completion = client.chat.completions.create(
            model="o1-mini-2024-09-12",
            messages=[
               # {"role": "system", "content": "Generate exactly one line of poetry based on the prompt."},
                {
                    "role": "user", 
                    "content": "Reply with only the line requested."+ prompt_dict['prompt']}
            ]
        )
        
        # Extract the generated line and add it to the dictionary
        print(completion.choices[0].message.content.strip())
        prompt_dict['response'] = completion.choices[0].message.content.strip()
        gen_prompt_dict.append(prompt_dict)
        
    except Exception as e:
        print(f"Error processing prompt: {prompt_dict['prompt']}")
        print(f"Error message: {str(e)}")
        prompt_dict['response'] = "Error generating response"



In [2]:
df_prompts = pd.read_csv('/Users/bsg25/metricalgpt/saved_corpora_scansions_results/gemini/gemini_responses.csv') # replace file path with your own


In [ ]:
## this cell will parse the above results, 
# but it is recommended to use parse_analysis/prosodic_vs_metricalbert.ipynb 
# or scansion_scripts/easy_prosodic_scansion_tool.ipynb

import pandas as pd
import prosodic
prosodic.logger.setLevel('ERROR')

# Convert the list of dictionaries to a DataFrame
#df_prompts = pd.DataFrame(gen_prompt_dict)

# Display the DataFrame
#df_prompts

# Set meter. Defaults here force iambic pentameter and prioritize the rule against lexically "strong" peaks (w_peak)"
# try other configurations to align with your sense of the meter!

constraints = {
    'w_peak': 2.0,
    'w_stress': 1.0,
    's_unstress': 1.0,
    'unres_across': 1.0,
    'unres_within': 0.0,
    'pentameter': 10.0,
    'iambic': 10.0,
}
meter = prosodic.Meter(
    constraints=constraints,
    resolve_optionality=True,
    max_s=1,
    max_w=2,
)

# Create an empty DataFrame with the desired column headers
column_headers = ["line_txt", "txt", "meter", "stress", "score", "num_viols", "ambig"]
df = pd.DataFrame(columns=column_headers)

# Loop through the "response" column of prompt_responses and add data to DataFrame
for response in df_prompts['response']:
    response = response.strip()  # Remove leading/trailing whitespace

    response = prosodic.Text(response)  # Process with prosodic.Text()
    parses = response.parse()
    parse_dict = parses[0].best_parse.attrs

    # Extract data from the first parse (modify if you need data from other parses)
    new_dict = {key: parse_dict[key] for key in column_headers if key in parse_dict}

    # Append data to DataFrame
    df = pd.concat([df, pd.DataFrame([new_dict])], ignore_index=True)

# Merge the new DataFrame with the original prompt_responses DataFrame
merged_df = df_prompts.merge(df, left_on='response', right_on='line_txt', how='left')
# Display the merged DataFrame

merged_df
merged_df = merged_df.drop_duplicates()

# Save the merged DataFrame to a CSV file [replace path]
merged_df.to_csv('openai_parsed_to_analyze.csv', index=False)  # Replace with your output file path

In [ ]:
merged_df

In [ ]:
#see available openAI models

import os
import openai

client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_KEY")
)
models = client.models.list()
for model in models:
    print(model.id)

The cells below are for a new run of prompts and parses!

In [42]:
#openai api
#%pip install openai

import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_KEY"),  # replace with your API key
)

# Assuming you have set your OpenAI API key

for prompt_dict in all_prompt_dicts:
    try:
        # Create completion using OpenAI API
        completion = client.chat.completions.create(
            model="chatgpt-4o-latest",
            messages=[
               # {"role": "system", "content": "Generate exactly one line of poetry based on the prompt."},
                {
                    "role": "user", 
                    "content": "Reply with only the line of poetry requested."+ prompt_dict['prompt']}
            ]
        )
        
        # Extract the generated line and add it to the dictionary
        print(completion.choices[0].message.content.strip())
        prompt_dict['response'] = completion.choices[0].message.content.strip()
        all_prompt_dicts.append(prompt_dict)
        
    except Exception as e:
        print(f"Error processing prompt: {prompt_dict['prompt']}")
        print(f"Error message: {str(e)}")
        prompt_dict['response'] = "Error generating response"



The forest whispers soft, where dwells the bear.
The mighty whale glides through the ocean deep.
The gentle sloth descends the branch with care.
The slender cat now prowls beneath the moon.
The loyal dog does wait beneath the moon.
Beneath the sun, the graceful giraffe strides.
The graceful leap, a gazelle in the night.
The cunning baboon danced beneath the moon.
Beneath the moon, I spied a sly raccoon.
The vibrant macaw sings beneath the skies.
The soft alpaca roams the mountain high.
The shadowed hyena prowls beneath the moon.
A quiet chinchilla dreams beneath the moon.
The mournful cries of midnight’s lone coyote.
The fearful gorilla hides deep in the shade.
Upon the plains, I spied a kangaroo.
A curious platypus glides through the stream.
Beware the quiet steps of porcupine.
Beneath the moon, the pangolin doth crawl.
Beneath the moon, the ocelot does prowl.
The lemur leaps through trees with grace unseen.
The lemming leaps into the restless tide.
The playful otter glides through st

KeyboardInterrupt: 

In [ ]:
import pandas as pd



# Convert the prompt_dict to a DataFrame
df_prompt_dict = pd.DataFrame(prompt_dict)
df_prompt_dict = df_prompt_dict.fillna("None")
df_prompt_dict.to_csv('prompt_dict.csv', index=False)


In [52]:
import pandas as pd
import prosodic

# Read the "response" column of "prompt_responses" and add the output to the CSV

prosodic.logger.setLevel('ERROR')


# Set meter. Defaults here force iambic pentameter and prioritize the rule against lexically "strong" peaks (w_peak)"
# try other configurations to align with your sense of the meter!

constraints = {
    'w_peak': 2.0,
    'w_stress': 1.0,
    's_unstress': 1.0,
    'unres_across': 1.0,
    'unres_within': 0.0,
    'pentameter': 10.0,
    'iambic': 10.0,
}
meter = prosodic.Meter(
    constraints=constraints,
    resolve_optionality=True,
    max_s=1,
    max_w=2,
)

# Create an empty DataFrame with the desired column headers
column_headers = ["line_txt", "txt", "meter", "stress", "score", "num_viols", "ambig"]
df = pd.DataFrame(columns=column_headers)

# Loop through the "response" column of prompt_responses and add data to DataFrame
for response in df_4o_latest['response']:
    response = response.strip()  # Remove leading/trailing whitespace

    response = prosodic.Text(response)  # Process with prosodic.Text()
    parses = response.parse()
    parse_dict = parses[0].best_parse.attrs

    # Extract data from the first parse (modify if you need data from other parses)
    new_dict = {key: parse_dict[key] for key in column_headers if key in parse_dict}

    # Append data to DataFrame
    df = pd.concat([df, pd.DataFrame([new_dict])], ignore_index=True)

# Merge the new DataFrame with the original prompt_responses DataFrame
df_4o_latest_merged = df_4o_latest.merge(df, left_on='response', right_on='line_txt', how='left')
# Display the merged DataFrame

# Save the merged DataFrame to a CSV file [replace path]
#merged_df.to_csv('/FILEPATH, index=False)  # Replace with your output file path

[3632.81s] Parsing lineparts [6x]:   0%|          | 0/1 [00:00<?, ?it/s]

































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [54]:
# Add "none" if variation is blank, then delete rows with "missing value"
df_4o_latest_merged['variation'] = df_4o_latest_merged['variation'].replace('', 'none')
df_4o_latest_merged = df_4o_latest_merged.dropna()

In [ ]:
df_4o_latest_merged.to_csv('/Users/bsg25/metricalgpt/merged_to_analyze_4o_latest.csv')